# Referece Class 1: US Gasoline Prices (2022 Q1 - 2025 Q1)

In [1]:
import pandas as pd
import requests

## Fule Prices from EIA

In [2]:
# API endpoint from the EIA
url = "https://api.eia.gov/v2/petroleum/pri/gnd/data/"

# Parameters for the request
params = {
    "frequency": "monthly",
    "data[0]": "value",
    "facets[series][]": "EMM_EPM0_PTE_NUS_DPG",
    "start": "2019-04",
    "end": "2025-05",
    "sort[0][column]": "period",
    "sort[0][direction]": "desc",
    "offset": 0,
    "length": 5000,
    "api_key": "cfDUIIZIGiRD1lQfAdTQGvkqM3Kf18nYKp5RJb1B"
}

# Make the request
response = requests.get(url, params=params)
data = response.json()

# Extract data into a DataFrame
records = data["response"]["data"]
df = pd.DataFrame(records)

In [3]:
df.head()

,period,duoarea,area-name,product,product-name,process,process-name,series,series-description,value,units
0,2025-04,NUS,U.S.,EPM0,Total Gasoline,PTE,Retail Sales,EMM_EPM0_PTE_NUS_DPG,U.S. All Grades All Formulations Retail Gasoli...,3.299,$/GAL
1,2025-03,NUS,U.S.,EPM0,Total Gasoline,PTE,Retail Sales,EMM_EPM0_PTE_NUS_DPG,U.S. All Grades All Formulations Retail Gasoli...,3.223,$/GAL
2,2025-02,NUS,U.S.,EPM0,Total Gasoline,PTE,Retail Sales,EMM_EPM0_PTE_NUS_DPG,U.S. All Grades All Formulations Retail Gasoli...,3.247,$/GAL
3,2025-01,NUS,U.S.,EPM0,Total Gasoline,PTE,Retail Sales,EMM_EPM0_PTE_NUS_DPG,U.S. All Grades All Formulations Retail Gasoli...,3.196,$/GAL
4,2024-12,NUS,U.S.,EPM0,Total Gasoline,PTE,Retail Sales,EMM_EPM0_PTE_NUS_DPG,U.S. All Grades All Formulations Retail Gasoli...,3.139,$/GAL


In [4]:
# Aggregate QoQ fule prices
# Convert 'period' to datetime
df['period'] = pd.to_datetime(df['period'])

# Convert 'value' to numeric
df['value'] = pd.to_numeric(df['value'], errors='coerce')

# Extract year and quarter
df['year'] = df['period'].dt.year
df['quarter'] = df['period'].dt.quarter
df['year_quarter'] = df['year'].astype(str) + "." + df['quarter'].astype(str)

# Group by year_quarter and average the gasoline price
df_grouped = df.groupby('year_quarter')['value'].mean().reset_index()

In [5]:
df_grouped

,year_quarter,value
0,2019.2,2.877000
1,2019.3,2.737000
2,2019.4,2.687333
3,2020.1,2.499333
4,2020.2,2.023000
5,2020.3,2.272667
6,2020.4,2.244000
7,2021.1,2.635000
8,2021.2,3.060333
9,2021.3,3.252667


## Tesla Production Data

In [6]:
tesla_df = pd.read_csv("/Users/zhang/Downloads/Stat 165 Final Project Workbook - Sheet2 (1).csv")
tesla_df["Total Production"] = tesla_df["Total Production"].str.replace(",", "").astype(int)
tesla_df["Total Deliveries"] = tesla_df["Total Deliveries"].str.replace(",", "").astype(int)

# Create year_quarter field for merging
tesla_df["year_quarter"] = tesla_df["Year"].astype(str) + "." + tesla_df["Quarter"].astype(str)

# Preview
tesla_df.head()

,Year,Quarter,Total Production,Total Deliveries,Delivery-Production Ratio,year_quarter
0,2019,2,87048,95200,1.093649,2019.2
1,2019,3,96155,97000,1.008788,2019.3
2,2019,4,104891,112000,1.067775,2019.4
3,2020,1,102672,88496,0.861929,2020.1
4,2020,2,82272,90891,1.104762,2020.2


In [7]:
# Ensure value is numeric and period formatting matches
df_grouped['value'] = pd.to_numeric(df_grouped['value'], errors='coerce')

## Analysis

In [8]:
# merge fuel price & Tesla productiond data
merged = tesla_df.merge(df_grouped, on='year_quarter', how='inner')
merged.rename(columns={'value': 'gas_price', 'Total Production': 'production'}, inplace=True)

# Two regimes for fuel prices >= or < 3.5
high_price_class = merged[merged['gas_price'] >= 3.5]
low_price_class = merged[merged['gas_price'] < 3.5]
mean_high = high_price_class['production'].mean()
mean_low = low_price_class['production'].mean()

# Assign weights (95% weight to high price conditions because we beleive prices will remain elevated)
forecast = 0.95 * mean_high + 0.05 * mean_low

mean_high, mean_low, forecast

(384375.71428571426, 258192.76470588235, 378066.56680672267)

# Reference Class 2: Historical Quarter-over-Quarter Growth (2019 Q4 - 2025 Q1)

In [9]:
# Pivot the dataframe to have quarters as columns and years as rows
pivot = tesla_df.pivot(index='Year', columns='Quarter', values='Total Production')

# Keep only the years with all four quarters present
valid_years = pivot.dropna()

# Compute average quarter-over-quarter growths
q1_q2_growth = (valid_years[2] - valid_years[1]) / valid_years[1]
q2_q3_growth = (valid_years[3] - valid_years[2]) / valid_years[2]
q3_q4_growth = (valid_years[4] - valid_years[3]) / valid_years[3]

# Compute average growth rates
avg_q1_q2 = q1_q2_growth.mean()
avg_q2_q3 = q2_q3_growth.mean()
avg_q3_q4 = q3_q4_growth.mean()

# Start with known Q1 2025 production
q1_2025 = tesla_df[(tesla_df["Year"] == 2025) & (tesla_df["Quarter"] == 1)]["Total Production"].values[0]

# Forecast Q2, Q3, Q4 2025
q2_2025 = q1_2025 * (1 + avg_q1_q2)
q3_2025 = q2_2025 * (1 + avg_q2_q3)
q4_2025 = q3_2025 * (1 + avg_q3_q4)

# Display results
print(f"Avg Q1→Q2 growth: {avg_q1_q2:.2%}")
print(f"Avg Q2→Q3 growth: {avg_q2_q3:.2%}")
print(f"Avg Q3→Q4 growth: {avg_q3_q4:.2%}")
print(f"\nForecasted Q2 2025: {q2_2025:,.0f}")
print(f"Forecasted Q3 2025: {q3_2025:,.0f}")
print(f"Forecasted Q4 2025: {q4_2025:,.0f}")

Avg Q1→Q2 growth: -3.42%
Avg Q2→Q3 growth: 27.42%
Avg Q3→Q4 growth: 17.10%

Forecasted Q2 2025: 350,202
Forecasted Q3 2025: 446,232
Forecasted Q4 2025: 522,521
